In [1]:
import numpy as np
import webbrowser
from aocd.models import Puzzle
import re
from dataclasses import dataclass
import json
from itertools import combinations
from collections import defaultdict
from typing import Optional
from scipy.spatial.distance import cdist
%matplotlib inline
puzzle = Puzzle(year=2021, day=19)

In [3]:
webbrowser.open(puzzle.url);

## Part 1

In [2]:
input_data = """
--- scanner 0 ---
0,2
4,1
3,3

--- scanner 1 ---
-1,-1
-5,0
-2,1
""".strip()

In [3]:
input_data = """
--- scanner 0 ---
404,-588,-901
528,-643,409
-838,591,734
390,-675,-793
-537,-823,-458
-485,-357,347
-345,-311,381
-661,-816,-575
-876,649,763
-618,-824,-621
553,345,-567
474,580,667
-447,-329,318
-584,868,-557
544,-627,-890
564,392,-477
455,729,728
-892,524,684
-689,845,-530
423,-701,434
7,-33,-71
630,319,-379
443,580,662
-789,900,-551
459,-707,401

--- scanner 1 ---
686,422,578
605,423,415
515,917,-361
-336,658,858
95,138,22
-476,619,847
-340,-569,-846
567,-361,727
-460,603,-452
669,-402,600
729,430,532
-500,-761,534
-322,571,750
-466,-666,-811
-429,-592,574
-355,545,-477
703,-491,-529
-328,-685,520
413,935,-424
-391,539,-444
586,-435,557
-364,-763,-893
807,-499,-711
755,-354,-619
553,889,-390

--- scanner 2 ---
649,640,665
682,-795,504
-784,533,-524
-644,584,-595
-588,-843,648
-30,6,44
-674,560,763
500,723,-460
609,671,-379
-555,-800,653
-675,-892,-343
697,-426,-610
578,704,681
493,664,-388
-671,-858,530
-667,343,800
571,-461,-707
-138,-166,112
-889,563,-600
646,-828,498
640,759,510
-630,509,768
-681,-892,-333
673,-379,-804
-742,-814,-386
577,-820,562

--- scanner 3 ---
-589,542,597
605,-692,669
-500,565,-823
-660,373,557
-458,-679,-417
-488,449,543
-626,468,-788
338,-750,-386
528,-832,-391
562,-778,733
-938,-730,414
543,643,-506
-524,371,-870
407,773,750
-104,29,83
378,-903,-323
-778,-728,485
426,699,580
-438,-605,-362
-469,-447,-387
509,732,623
647,635,-688
-868,-804,481
614,-800,639
595,780,-596

--- scanner 4 ---
727,592,562
-293,-554,779
441,611,-461
-714,465,-776
-743,427,-804
-660,-479,-426
832,-632,460
927,-485,-438
408,393,-506
466,436,-512
110,16,151
-258,-428,682
-393,719,612
-211,-452,876
808,-476,-593
-575,615,604
-485,667,467
-680,325,-822
-627,-443,-432
872,-547,-609
833,512,582
807,604,487
839,-516,451
891,-625,532
-652,-548,-490
30,-46,-14
""".strip()

In [4]:
input_data = puzzle.input_data

In [5]:
def read_scanners(input_data):
    scanners = input_data.split("\n\n")
    scanners = [s.split("\n")[1:] for s in scanners]
    scanners = [np.asarray([np.fromiter(map(int, l.split(",")), int) for l in c]) for c in scanners]
    return scanners

### Rotations

In [6]:
scanner = read_scanners("""
--- scanner 0 ---
-1,-1,1
-2,-2,2
-3,-3,3
-2,-3,1
5,6,-4
8,0,7
""".strip())[0]

In [7]:
x, y, z = 1, 2, 3  # don't use 0, 1, 2, since 0 can't encode a sign as well
rotations = np.asarray([
    # 4 rotations facing positive X
    [x, y, z],
    [x, -z, y],
    [x, -y, -z],
    [x, z, -y],
    # 4 rotations facing negative X
    [-x, y, -z],
    [-x, -z, -y],
    [-x, -y, z],
    [-x, z, y],
    # 4 rotations facing positive Y
    [-y, x, z],
    [-z, x, -y],
    [y, x, -z],
    [z, x, y],
    # 4 rotations facing negative Y
    [y, -x, z],
    [-z, -x, y],
    [-y, -x, -z],
    [z, -x, -y],
    # 4 rotations facing positive Z
    [-z, y, x],
    [-y, -z, x],
    [z, -y, x],
    [y, z, x],
    # 4 rotations facing negative Z
    [z, y, -x],
    [y, -z, -x],
    [-z, -y, -x],
    [-y, z, -x],
])
assert len(rotations) == 24

In [8]:
def rotate(points, rotation):
    # make use more convenient for just a single point
    single_point = False
    if points.ndim == 1:
        single_point = True
        points = points[np.newaxis, :]
    rotated = np.zeros_like(points)
    signed = points * np.sign(rotation)
    axes_order = np.abs(rotation) - 1  # revert to zero indexing
    rotated[:, 0] = signed[:, axes_order[0]]
    rotated[:, 1] = signed[:, axes_order[1]]
    rotated[:, 2] = signed[:, axes_order[2]]
    if single_point:
        return rotated[0]
    return rotated

def inverse_rotate(points, rotation):
    # make use more convenient for just a single point
    single_point = False
    if points.ndim == 1:
        single_point = True
        points = points[np.newaxis, :]
    axes_order = np.abs(rotation) - 1  # revert to zero indexing
    rotated = np.zeros_like(points)
    rotated[:, axes_order[0]] = points[:, 0]
    rotated[:, axes_order[1]] = points[:, 1]
    rotated[:, axes_order[2]] = points[:, 2]
    rotated = rotated * np.sign(rotation)
    if single_point:
        return rotated[0]
    return rotated

def all_rotations(points):
    for rotation in rotations:
        yield rotate(points, rotation), rotation

In [9]:
scanners = read_scanners(input_data)

In [10]:
def calc_pairwise_point_differences(points):
    # calculate the difference between each pair of points
    diffs = {}
    for i, j in combinations(range(len(points)), 2):
        diff = tuple(points[i] - points[j])
        diffs[diff] = (i, j)
    # diffs is a mapping from a 3-tuple (dx, dy, dz) to indices of the two points where this difference occurs
    return diffs

In [11]:
known_rotations = {
    0: rotations[0]  # we define scanner 0 as the base rotation. Then this dict will later contain the rotation we need to apply to every scanner to rotate it the same way as scanner 0
}

known_positions = {
    0: np.zeros(3, int),  # known positions of all scanners relative to scanner 0
}

In [106]:
def find_matching_beacons(scanner0, scanner1):
    points0 = rotate(scanners[scanner0], known_rotations[scanner0])  # always rotate all the points to the rotation of scanner 0
    diffs0 = calc_pairwise_point_differences(points0)
    for points1, rotation in all_rotations(scanners[scanner1]):
        diffs1 = calc_pairwise_point_differences(points1)
        # find differences that occur for both scanners:
        match = set(diffs0.keys()) & set(diffs1.keys())
        # based on those differences find points that occur for both scanners
        point_mapping = {}
        for dist in match:
            indices_a, indices_b = diffs0[dist], diffs1[dist]
            point_mapping[indices_a[0]] = set(indices_b)
            point_mapping[indices_a[1]] = set(indices_b)

        for dist in match:
            indices_a, indices_b = diffs0[dist], diffs1[dist]
            for i in range(2):
                candidates = point_mapping[indices_a[i]]
                if isinstance(candidates, int):  # already determined the matching point in Scanner B for this point in Scanner A
                    continue
                found = candidates & set(indices_b)
                if len(found) == 1:
                    point_mapping[indices_a[i]], = found
        
        if len(point_mapping) >= 5:  # should be 12, but then no overlap is found for the last 3 scanners for my input :( - but works with 5 also :D
            i, j = next(iter(point_mapping.items()))  # we only need one point to calculate the position of scanner1 relative to scanner0
            found_scanner = points0[i] - points1[j]  # position of scanner1 relative to scanner0
            found_beacons = points0[list(point_mapping.keys())]
            return found_scanner, rotation, found_beacons
    return None, None, None

In [107]:
# May take a while:
remaining = True
while remaining:
    remaining = set(range(len(scanners))) - known_positions.keys()  # all the scanners we still have to locate
    for i in list(known_positions.keys()):
        for j in remaining:
            if i == 7 and j == 4:
                print("now")
            found_scanner, rotation, found_beacons = find_matching_beacons(i, j)
            if found_scanner is None:
                continue
            known_positions[j] = known_positions[i] + found_scanner
            known_rotations[j] = rotation

#### Now we know all the scanner positions and rotations, so we can easily assemble a list of beacons

In [109]:
beacons = set()
for i in range(len(scanners)):
    if i not in known_positions:
        continue
    points = known_positions[i] + rotate(scanners[i], known_rotations[i])
    for p in points:
        beacons.add(tuple(p))

In [110]:
result = len(beacons)
result

353

In [111]:
puzzle.answer_a = result

That's the right answer!  You are one gold star closer to finding the sleigh keys. [Continue to Part Two]


## Part 2

In [117]:
max_dist = -np.inf
for scanner_a, scanner_b in combinations(known_positions.values(), 2):
    dist = np.abs(scanner_a - scanner_b).sum()
    if dist > max_dist:
        max_dist = dist

In [118]:
result = max_dist
result

10832

In [119]:
puzzle.answer_b = result

That's the right answer!  You are one gold star closer to finding the sleigh keys.You have completed Day 19! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
